<a href="https://colab.research.google.com/github/AravaChoudhary/SVM/blob/main/sample_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
# Configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# using API to fetch the dataset
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 96% 78.0M/80.9M [00:02<00:00, 41.2MB/s]
100% 80.9M/80.9M [00:02<00:00, 31.0MB/s]


In [ ]:
# extracting the compressed dataset from zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset , 'r') as zip:
  zip.extractall()
  print('The Dataset is Extracted')

The Dataset is Extracted


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
import nltk
nltk.download('stopwords')

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Data Processing
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , encoding = 'ISO-8859-1') # Corrected encoding name


In [72]:
twitter_data.shape

(1600000, 6)

In [73]:
twitter_data.head(4)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


In [74]:
# naming the columns and reading the dataset again
column_names = ['target','id','date','flag','user','text']

twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , names = column_names , encoding = 'ISO-8859-1')
print(twitter_data.shape)
twitter_data.head()

(1600000, 6)


,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [75]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [76]:
twitter_data.target.value_counts()

,count
target,
0,800000
4,800000


In [77]:
# converting 4 --> 1
twitter_data.replace({'target' : {4:1}}, inplace = True)
twitter_data.target.value_counts()

,count
target,
0,800000
1,800000


In [78]:
# Stemming
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

port_stem = PorterStemmer()

def stemming(text):
  stemmed_text = re.sub('[^a-zA-Z]',' ',text)
  stemmed_text = stemmed_text.lower()
  stemmed_text = stemmed_text.split()
  stemmed_text = [port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english')]
  stemmed_text = ' '.join(stemmed_text)

  return stemmed_text

In [112]:
df_label_0 = twitter_data[twitter_data['target'] == 0]
df_label_1 = twitter_data[twitter_data['target'] == 1]

df_sample_0 = df_label_0.sample(n=8000, random_state=1)
df_sample_1 = df_label_1.sample(n=8000, random_state=1)

df_balanced_sample = pd.concat([df_sample_0, df_sample_1])
df_balanced_sample = df_balanced_sample.sample(frac=1, random_state=1).reset_index(drop=True)

df_balanced_sample.target.value_counts()
sample_data = df_balanced_sample.copy()
sample_data.target.value_counts()

,count
target,
0,8000
1,8000


In [106]:
sample_data = twitter_data.sample(frac=0.01, random_state=1)
sample_data

,count
target,
1,8018
0,7982


In [113]:
sample_data['stemmed_text'] = sample_data['text'].apply(stemming)

In [114]:
sample_data['stemmed_text']

,stemmed_text
0,shave two year life one morn also realis tweet...
1,school stuff want sunni place earth ice cream
2,gettin hair
3,realli feel like hula hoop right pitti took ma...
4,hungri noth fridg caesar salad unsatisfi
...,...
15995,rustedhors mine sore also damn time month
15996,noth listenin music bore austin almost die day
15997,kid fun wore xd could use stori right sleep la...
15998,ddlovato talk drink tea honey sooo sowwi ur voic


In [115]:
X = sample_data['stemmed_text'].values
y = sample_data['target'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 2)
X.shape , X_train.shape , X_test.shape

((16000,), (12800,), (3200,))

In [116]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [117]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)


rf_train_accuracy = accuracy_score(y_train, rf_model.predict(X_train))
rf_test_accuracy = accuracy_score(y_test, rf_model.predict(X_test))

print("Current Model Performance : ")
print(f"Random Forest - Training Accuracy: {rf_train_accuracy * 100:.2f}%")
print(f"Random Forest - Testing Accuracy: {rf_test_accuracy * 100:.2f}%")




Random Forest - Training Accuracy: 99.95%
Random Forest - Testing Accuracy: 73.03%


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Define parameter distributions
param_distributions = {
    'n_estimators': randint(50, 500),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5),
    'bootstrap': [True, False]
}

# Initialize RandomForestClassifier
rf = RandomForestClassifier()

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions,n_iter=50, cv=5, verbose=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Print best parameters and best score
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best Score: {random_search.best_score_:.2f}")

# Use the best estimator from random search
best_rf = random_search.best_estimator_

# Predict and evaluate
train_accuracy = accuracy_score(y_train, best_rf.predict(X_train))
test_accuracy = accuracy_score(y_test, best_rf.predict(X_test))

print(f"Random Forest - Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Random Forest - Testing Accuracy: {test_accuracy * 100:.2f}%")


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
